In [1]:
from SALib.sample import saltelli
from SALib.sample import sobol_sequence 
from SALib import ProblemSpec
from SALib.analyze import sobol

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from tqdm import tqdm
import diptest 

## Sensitivity analysis functions

#### Generating problem specification and samples

In [2]:
def sp_and_samples(names, bounds, dists, outputs, int_list, N):
       sp = ProblemSpec({
              "names": names,
              "groups": None,
              "bounds": bounds,           
              "dists": dists,  
              "outputs": outputs,
              })

       sp.sample_sobol(N, calc_second_order=True)
       X = sp.samples
       n_samples = np.shape(X)[0]

       # ne must be an integer value
       sample_dict = {i:(X[:,id] if int_list[id] == 0 else (np.rint(X[:,id])).astype(int)) for id, i in enumerate(names)}

       # find duplicates in the sample space to speed up the process
       u, c = np.unique(X, return_counts=True, axis=0)
       dup = u[c > 1]
       dup_dict = {str([j for j in dup[i,:]]):[0,0,0,0] for i in range(np.shape(dup)[0])}

       return sp, n_samples, sample_dict, dup_dict

#### Running the model

In [1]:
def SA_results(sample_dict, n_samples, dup_dict, x, n, z, P, ne, d, T, e1, d0, 
               impact, sigma, b1, b2, b3, h_upd_chance, eta, choice_intensity, a1l=2, a1h=2, a2l=2, a2h=2, a3l=2, a3h=2):
    results = np.zeros((n_samples, 2)) # 2 for system rationality and bimodality

    def SA_run_sample(x, n, z, P, ne, d, T, e1, d0, a1l, a1h, a2l, a2h, a3l, a3h, impact, sigma, b1, b2, b3, h_upd_chance, eta, choice_intensity):
        env_results = np.zeros(x)
        sys_rat_results = np.zeros(x)
        nash_env1, nash_zl1, nash_env2, nash_zl2 = nash_eq(d0, a1l, a1h, a2l, a2h, a3l, a3h, e1, P, d, n, T)

        for i in range(x):
            n_evolution, z_evolution, zl_predictions, pd, heuristic_fraction, observed_z, acting_p, degrees, skew \
                = model(n, z, P, ne, T, e1, d0, a1l, a1h, a2l, a2h, a3l, a3h, impact, sigma, b1, b2, b3, h_upd_chance, eta, choice_intensity)
            env_results[i] = n_evolution[-1] # use the final period to calculate Dip test statistic 
            sys_rat_results[i] = np.max([float(ave_kl(z_evolution, nash_zl1)), float(ave_kl(z_evolution, nash_zl2))])
        sys_rat = np.mean(sys_rat_results) 
        dip, pval = diptest.diptest(env_results) # use p-value for bimodality
                        
        return sys_rat, pval

    for i in tqdm(range(n_samples)):
        values = [sample_dict[j][i] for j in sample_dict.keys()]
        [d0, a1d, a2d, a3d, ne, impact] = values
        a1h, a2h, a3h = (a1l + a1d, a2l + a2d, a3l + a3d)
        key = str([j for j in values])
        if key in dup_dict.keys():
            if dup_dict[key][0] > 0:
                results[i,:] = dup_dict[key][1:]
            else: 
                sys_rat, bimodality = SA_run_sample(x, n, z, P, ne, d, T, e1, d0, a1l, a1h, a2l, a2h, a3l, a3h, 
                                                                         impact, sigma, b1, b2, b3, h_upd_chance, eta, choice_intensity)
                results[i,:] = [sys_rat, bimodality]
                dup_dict[key][1:] = [sys_rat, bimodality]
            dup_dict[key][0] += 1
        else: 
            sys_rat, bimodality = SA_run_sample(x, n, z, P, ne, d, T, e1, d0, a1l, a1h, a2l, a2h, a3l, a3h, 
                                                                     impact, sigma, b1, b2, b3, h_upd_chance, eta, choice_intensity)
            results[i,:] = [sys_rat, bimodality]

    return results 

#### Plot results

In [ ]:
def plot_sa_results(sp):
    # organizing the sensitivity indices
    num_vars = sp['num_vars']
    num_outputs = len(sp['outputs'])
    S = np.zeros((2, num_outputs, 2, num_vars))   # 2 first because of ST S1; 2 second because one for estimate and one for confidence interval

    for id, x in enumerate(sp['outputs']):
        S[0,id,:,:] = [sp.analysis[x]['ST'], sp.analysis[x]['ST_conf']]
        S[1,id,:,:] = [sp.analysis[x]['S1'], sp.analysis[x]['S1_conf']]
    
    def plotting(ST_row, S1_row, num_vars):
        y = np.arange(1, num_vars*3, 3)
        
        def per_order(y, S_row, color):
            ub = S_row[0,:] + S_row[1,:]
            lb = S_row[0,:] - S_row[1,:]
            if color == 'red':
                plt.scatter(y, S_row[0,:], c=color, label='ST') 
            else:
                plt.scatter(y, S_row[0,:], c=color, label='S1') 
            for id, x in enumerate(y):
                plt.plot([x, x], [lb[id], ub[id]], color='black', linestyle='dashed')
        per_order(y-1, ST_row, 'red')
        per_order(y, S1_row, 'blue')
        plt.grid()
        plt.legend(fontsize=20)
        plt.xticks(y-0.5, [r'$\alpha$', r'$\gamma_{1}^{d}$', r'$\gamma_{2}^{d}$', r'$\gamma_{3}^{d}$', r'$\lambda$', r'$\nu$'], fontsize=20)
        plt.yticks(fontsize=20)

    # plotting the sensitivity indices
    fig = plt.figure(figsize=(20, 8)) #20,8
    rows = 1
    columns = num_outputs
    count = 1

    for id, i in enumerate(['System rationality', 'Bimodality']):
        fig.add_subplot(rows, columns, count)
        plotting(S[0,id,:,:], S[1,id,:,:], num_vars)
        plt.ylabel(f'{i}', fontsize=20)
        count += 1
    
    plt.tight_layout()
    plt.savefig('sensitivity_analysis_paper/plots/SA_plot.png', dpi=300)
    plt.show()